In [1]:
#import pandas
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import re as re
import os
import requests  # library for accessing content from web URLs
import pprint  # library for making Python data structures readable
pp = pprint.PrettyPrinter()
import json    # library for working with JSON-formatted text strings

In [2]:
#set directory for one file not available through API
raw = '/Users/briangoggin/Dropbox/CP 255/Permit Project/Raw Data/'

d2014 = 'https://data.sfgov.org/resource/b8d6-zthg.json'
d2013 = 'https://data.sfgov.org/resource/sjse-8gyy.json'
d2012 = 'https://data.sfgov.org/resource/a64c-96a5.json'
d2011 = 'https://data.sfgov.org/resource/pwiv-ej3p.json'

In [3]:
#import 2015
df15 = pd.read_excel(raw+'Housing Inventory/2015_datasf.xlsx', sheetname = '2015Completes')

In [4]:
#define function for importing other data

def importdata(year, yearvalue, field1, field2, field3, field4, field5, field6, field7, field8, field9):
    '''
    This function calls the API endpoint quarter and returns a dataframe with the desired columns.
    field1=geogfield2 represent the desired fields as they are named at the API endpoint
    '''
    
    def includekey(field):
        '''
        This function takes a given field from the API endpoint and creates a list out of its entires.
        '''
        list = []
        for item in data: 
            if field in item.keys():
                list.append(item[field])
            else:
                list.append(np.nan)
        return list
    
    response = requests.get(year)
    results = response.text
    data = json.loads(results) #data is a list at this point
    
    #import fields
    d = {}
    d['block'] = includekey(field1)
    d['lot'] = includekey(field2)
    d['address'] = includekey(field3)
    d['status'] = includekey(field4)
    d['latest_date'] = includekey(field5)
    d['units'] = includekey(field6)
    d['net_units'] = includekey(field7)
    d['affordable_units'] = includekey(field8)
    d['zone'] = includekey(field9)

    df = pd.DataFrame.from_dict(d)
    df['year'] = yearvalue
    
    return df

In [7]:
#import data
df2014 = importdata(d2014, '2014', 'block', 'lot', 'stdadd', 'action', 'actdate', 'units', 'netunits', 'affhsg', 'zoning')
df2013 = importdata(d2013, '2013', 'block', 'lot', 'standardaddress', 'change_type', 'actdate', 'units', 'netunits', 'aff_hsg', 'zoning')
df2012 = importdata(d2012, '2012', 'block', 'lot', 'standardaddress', 'change_type', 'actdate', 'units', 'netunits', 'aff_hsg', 'zoning')
df2011 = importdata(d2011, '2011', 'block', 'lot', 'standardad', 'change_type', 'actdate', 'units', 'netunits', 'affordable', 'zoning')


In [8]:
df2011.head()

,address,affordable_units,block,latest_date,lot,net_units,status,units,zone,year
0,365 FULTON ST,120,0792,NaN,028,120,NaN,120,NCT-3,2011
1,855 JAMESTOWN AV,NaN,4991,NaN,277,36,NaN,36,RH-2,2011
2,2139 OFARRELL ST,21,1101,NaN,022,21,NaN,21,RM-3,2011
3,420 29TH AV,20,1460,NaN,015,20,NaN,20,RH-2,2011
4,121 09TH ST,2,3728,NaN,069,20,NaN,20,SLR,2011


In [ ]:
#append data together
years = [df2014, df2013, df2012, df2011]
full_df = df2015.append(years)